# Homework 3

In [22]:
# !pip install cdlib
# !pip install leidenalg
# !pip uninstall community
# !pip install python-louvain
#!pip install igraph

import igraph
import infomap
import networkx as nx
from networkx.algorithms import node_classification
import random
from cdlib import algorithms, evaluation, NodeClustering
from community import community_louvain
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os

## 2. Who’s the winner?

### (i)

In [7]:
def build_synhtetic_graph(mu = 0.5):
    G = nx.Graph(name = "girvan_newman")
    n = 72
    cluster_div = 24
    for i in range(n):
        G.add_node(i, cluster = i // cluster_div + 1)
    for i in range(n):
        for j in range(i + 1, n):
            if G.nodes[i]['cluster'] == G.nodes[j]['cluster']:
                if random.random() < 20 * (1 - mu) / (cluster_div-1):
                    G.add_edge(i, j)
            else:
                if random.random() < 20 * mu / (n-cluster_div):
                    G.add_edge(i, j)
    return G

def compare_on_synthetic(algo_fn, mus, iter_num=10):
    nmis = []
    for mu in mus:
        NMI = 0
        for _ in range(iter_num):
            G = build_synhtetic_graph(mu)
            clustered_G = algo_fn(G)
            partition = get_graph_ideal_partition(G) 
            NMI += clustered_G.normalized_mutual_information(partition).score / iter_num
        nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis
        
def get_graph_ideal_partition(G):
    P = {}
    for node in G.nodes(data = True):
        if node[1]['cluster'] not in P:
            P[node[1]['cluster']] = []
        P[node[1]['cluster']].append(node[0])
    node_clusters = P.values()
    return NodeClustering(list(node_clusters), G, 'Ideal')

def run():
    iter_num = 25
    mus = [0.1 * i for i in range(0, 6)]
    algs = {
        "Louvain": algorithms.louvain,
        "Infomap": algorithms.infomap,
        "Walktrap": algorithms.walktrap
    }
    fig = plt.figure()
    plt.xlabel('µ')
    plt.ylabel('Normalized mutual information')
    plt.xticks(mus)

    for algo_name in algs:
        print('ALGORITHM:', algo_name)
        nmis = compare_on_synthetic(algs[algo_name], mus, iter_num)
        plt.plot(mus, nmis, label=algo_name)
        print('======================'*3)
        
    plt.savefig("images/2_i.jpg")
    plt.legend(loc="best")

run()

ALGORITHM: Louvain
mu=0.00, NMI: 1.000
mu=0.10, NMI: 1.000
mu=0.20, NMI: 1.000
mu=0.30, NMI: 0.998
mu=0.40, NMI: 0.953
mu=0.50, NMI: 0.389
ALGORITHM: (DC)SBM


TypeError: 'module' object is not callable

### (ii)

In [121]:
def read_net(folder, graph_name):
    file_name = graph_name + '.net'
    G = nx.MultiGraph(name = file_name)
    with open(os.path.join(folder, file_name), 'r', encoding='utf8') as f:
        f.readline()
        # add nodes
        for line in f:
            if line.startswith("*"):
                break
            else:
                node_info = line.split("\"")
                node = int(node_info[0]) - 1
                label = node_info[1]
                cluster = int(node_info[2]) if len(node_info) > 2 and len(node_info[2].strip()) > 0 else 0
                G.add_node(node, label=label, cluster=cluster)

        # add edges
        for line in f:
            node1_str, node2_str = line.split()[:2]
            G.add_edge(int(node1_str)-1, int(node2_str)-1)
    return G

def compare_on_lrf(algo_fn, mus, iter_num=10):
    nmis = []
    for mu in mus:
        NMI = 0
        for i in range(iter_num):
            G = read_net("data/LFR/", "LFR_0{}_{}".format(int(mu*10), i))
            clustered_G = algo_fn(G)
            partition = get_graph_ideal_partition(G) 
            NMI += clustered_G.normalized_mutual_information(partition).score / iter_num
            nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis
        
def get_graph_ideal_partition(G):
    P = {}
    for node in G.nodes(data = True):
        if node[1]['cluster'] not in P:
            P[node[1]['cluster']] = []
        P[node[1]['cluster']].append(node[0])
    node_clusters = P.values()
    return NodeClustering(list(node_clusters), G, 'Ideal')

iter_num = 25
mus = [0.2 * i for i in range(0, 5)]

for algo_name in algs:
    print('ALGORITHM:', algo_name)
    nmis = compare_on_lrf(algs[algo_name], mus, iter_num)
    print('======================'*3)  

ALGORITHM: Louvain
mu=0.00, NMI: 0.995
mu=0.20, NMI: 0.949
mu=0.40, NMI: 0.868
mu=0.60, NMI: 0.508
mu=0.80, NMI: 0.133
ALGORITHM: Walktrap
mu=0.00, NMI: 1.000
mu=0.20, NMI: 0.975
mu=0.40, NMI: 0.820
mu=0.60, NMI: 0.607
mu=0.80, NMI: 0.427
ALGORITHM: Label propagation
mu=0.00, NMI: 0.998
mu=0.20, NMI: 0.965
mu=0.40, NMI: 0.888
mu=0.60, NMI: 0.000
mu=0.80, NMI: 0.000


### (iii)

In [40]:
def compare_on_random(algo_fn, mus, iter_num=10):
    nmis = []
    for mu in mus:
        NMI = 0
        for i in range(iter_num):
            G = nx.gnm_random_graph(1000, 1000*mu)
            clustered_G = algo_fn(G)
            partition = NodeClustering([[r] for r in range(1000)], graph=None, method_name="rand_graph")
            NMI += clustered_G.variation_of_information(partition).score / iter_num
        nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis


iter_num = 25
mus = [8 * i for i in range(1, 6)]
for algo_name in algs:
    print('ALGORITHM:', algo_name)
    nmis = compare_on_random(algs[algo_name], mus, iter_num)
    print('======================'*3)

ALGORITHM: Louvain
mu=8.00, NMI: 7.072
mu=16.00, NMI: 6.692
mu=24.00, NMI: 6.820
mu=32.00, NMI: 6.826
mu=40.00, NMI: 6.772
ALGORITHM: Walktrap
mu=8.00, NMI: 5.902
mu=16.00, NMI: 6.582
mu=24.00, NMI: 6.778


KeyboardInterrupt: 

In [18]:
import math

def compare_on_random(algo_fn, mus, iter_num=10):
    nmis = []
    n = 1000
    for mu in mus:
        NMI = 0
        for i in range(iter_num):
            G = nx.gnm_random_graph(n, n/2*mu)
            clustered_G = algo_fn(G)
            partition = NodeClustering([[r] for r in range(n)], graph=G, method_name="rand_graph")
            NMI += clustered_G.variation_of_information(partition).score / math.log(n)
        nmis.append(NMI)
        print("mu={:.2f}, NMI: {:5.3f}".format(mu, NMI))
    return nmis


iter_num = 25
algs = {
    "Louvain": algorithms.louvain,
    "Infomap": algorithms.infomap,
    "Walktrap": algorithms.walktrap
}
mus = [8 * i for i in range(1, 6)]
for algo_name in algs:
    print('ALGORITHM:', algo_name)
    nmis = compare_on_random(algs[algo_name], mus, iter_num)
    print('======================'*3)

ALGORITHM: Louvain
mu=8.00, NMI: 22.276
mu=16.00, NMI: 25.462
mu=24.00, NMI: 24.529
mu=32.00, NMI: 24.444
mu=40.00, NMI: 24.378
ALGORITHM: Infomap


ModuleNotFoundError: Optional dependency not satisfied: install package wurlitzer to use infomap.

## 3. Peers, ties and the Internet

In [14]:
def leiden(G, node_pairs):
    coms = algorithms.leiden(G)
    C = dict()
    for i in range(len(coms.communities)):
        for node in coms.communities[i]:
            C[node] = i
    part_G = community_louvain.induced_graph(C, G)

    scores = []
    for n1, n2 in node_pairs:
        score = 0
        if C[n1] == C[n2]:
            try: 
                nc = len(coms.communities[C[n1]])
                mc = part_G[C[n1]][C[n2]]['weight']
                score = mc / (nc * (nc-1) / 2)
            except KeyError:
                score = 0
        scores.append([n1, n2, score])
    return scores

def sample(G, p):
    nodes = list(G.nodes())
    N = int(G.number_of_edges()*p)
    
    Ln = []
    while len(Ln) != N:
        node1, node2 = random.choice(nodes), random.choice(nodes)
        if not G.has_edge(node1, node2) and (node1, node2) not in Ln:
            Ln.append((node1, node2))
    
    Lp = random.sample(list(G.edges()), N)
    G.remove_edges_from(Lp)  
    
    return G, [*Ln, *Lp], len(Ln), N

def AUC(G, algs):
    G, nodes, l, N = sample(G.copy(), 0.1)

    scores = []
    for algo_name in algs:
        predicted = list(algs[algo_name](G, nodes))
        m1, m2 = 0, 0
        for n in range(N):
            s1 = random.sample(predicted[:l], 1)[0]
            s2 = random.sample(predicted[l:], 1)[0]
            if s2[2] > s1[2]:
                m1 += 1
            elif s1[2] == s2[2]:
                m2 +=1
        scores.append((m1 + m2/2) / N)
    return np.array(scores)

def AUC_runs(G, name, algs, n=10):
    auc_l, auc_p, auc_a = 0, 0, 0
    for i in range(n):
        scores = AUC(G, algs)
        auc_l += scores[0]
        auc_p += scores[1]
        auc_a += scores[2]
    print(name + ':')
    print("  Leiden:", auc_l/n)
    print("  Preferential attachment:", auc_p/n)
    print("  Adamic adar index:", auc_a/n)

In [15]:
algs = {
    "Leiden": leiden,
    "Pref attachment": nx.preferential_attachment,
    "Adamic adar index": nx.adamic_adar_index
}
# **********************************************************
G = nx.Graph(nx.read_pajek("data/gnutella.net")).to_undirected()
AUC_runs(G, "Gnutella", algs, 10)
# **********************************************************
G = nx.Graph(nx.read_pajek("data/circles.net")).to_undirected()
AUC_runs(G, "Facebook", 10)
# **********************************************************
G = nx.Graph(nx.read_pajek("data/nec.net")).to_undirected()
AUC_runs(G, "Internet", 10)
# **********************************************************
G = nx.gnm_random_graph(25000, 125000)
AUC_runs(G, "Random", algs, 10)

4553
58
3762
58
654
56
3762
58
4463
59
4463
59
13104
60
13104
60
13104
60
4505
58
4069
58
4553
58
4463
59
4069
58
4463
59
5081
59
4505
58
4463
59
4505
58
4069
58
13104
60
4463
59
13104
60
1066
56
4553
58
4463
59
13104
60
13104
60
4505
58
4505
58
13104
60
5081
59
4505
58
13104
60
5081
59
13104
60
13104
60
1106
57
13104
60
4463
59
3762
58
4553
58
3762
58
4505
58
13104
60
13104
60
13104
60
4463
59
13104
60
613
58
4463
59
13104
60
4069
58
5081
59
13104
60
13104
60
4505
58
13104
60
13104
60
4553
58
1095
58
4463
59
4499
59
13104
60
4069
58
849
58
777
56
13104
60
4505
58
3762
58
5081
59
13104
60
13104
60
13104
60
13104
60
5081
59
13104
60
4463
59
13104
60
13104
60
4069
58
4505
58
4553
58
4499
59
4069
58
4505
58
4499
59
4463
59
13104
60
4463
59
5081
59
4505
58
4499
59
13104
60
4499
59
777
56
5081
59
5081
59
372
53
722
57
4505
58
13104
60
13104
60
1095
58
3762
58
4463
59
13104
60
13104
60
3762
58
777
56
4553
58
972
57
5081
59
3762
58
13104
60
13104
60
486
56
5081
59
13104
60
5081
59
13104
60
37

13104
60
849
58
13104
60
13104
60
804
58
13104
60
13104
60
729
57
13104
60
4499
59
13104
60
13104
60
1095
58
1106
57
13104
60
4463
59
13104
60
681
57
13104
60
486
56
13104
60
13104
60
13104
60
5081
59
5081
59
13104
60
4463
59
13104
60
13104
60
4505
58
13104
60
13104
60
4499
59
13104
60
13104
60
13104
60
5081
59
4553
58
4553
58
13104
60
5081
59
5081
59
13104
60
4505
58
1155
58
4499
59
3762
58
13104
60
3762
58
4553
58
13104
60
1057
58
5081
59
777
56
13104
60
13104
60
14470
62
3918
61
4998
62
4998
62
4508
62
4998
62
547
60
4508
62
14470
62
14470
62
2672
61
14470
62
1173
61
14470
62
5926
62
14470
62
5926
62
4998
62
3918
61
4998
62
4985
62
5926
62
1115
61
14470
62
14470
62
3918
61
14470
62
14470
62
805
60
3918
61
14470
62
14470
62
4508
62
1115
61
3918
61
3918
61
5926
62
14470
62
5926
62
14470
62
3918
61
4508
62
5926
62
4998
62
14470
62
4508
62
14470
62
4998
62
527
61
14470
62
5926
62
5926
62
5926
62
14470
62
14470
62
14470
62
4985
62
959
61
714
60
3918
61
4508
62
959
61
4985
62
4998
62
1447

586
61
4508
62
4998
62
3918
61
4985
62
14470
62
14470
62
587
60
3918
61
14470
62
14470
62
14470
62
14470
62
14470
62
14470
62
14470
62
5926
62
1115
61
14470
62
680
60
14470
62
14470
62
14470
62
1238
60
2765
61
14470
62
5926
62
959
61
14470
62
14470
62
14470
62
3918
61
14470
62
14470
62
14470
62
14470
62
4508
62
5926
62
14470
62
680
60
4985
62
14470
62
2672
61
14470
62
4985
62
14470
62
2672
61
14470
62
4998
62
14470
62
4985
62
1238
60
14470
62
5926
62
14470
62
14470
62
14470
62
14470
62
959
61
14470
62
14470
62
3918
61
14470
62
14470
62
14470
62
643
61
14470
62
618
60
14470
62
14470
62
4998
62
595
59
14470
62
14470
62
5926
62
14470
62
14470
62
14470
62
4508
62
14470
62
959
61
14470
62
587
60
14470
62
1173
61
5926
62
5926
62
14470
62
14470
62
14470
62
4998
62
14470
62
2765
61
4998
62
14470
62
14470
62
4985
62
14470
62
4998
62
14470
62
14470
62
14470
62
14470
62
4998
62
14470
62
4985
62
4985
62
4998
62
859
61
14470
62
14470
62
14470
62
4998
62
1173
61
14470
62
4985
62
14470
62
14470
62
14

14309
61
1157
56
4471
56
4448
57
4471
56
4448
57
4448
57
1048
56
2221
56
1537
56
5202
58
14309
61
14309
61
14309
61
4448
57
1157
56
14309
61
1163
56
4471
56
14309
61
1537
56
4533
56
4448
57
4471
56
5202
58
2337
56
4517
56
4471
56
1095
56
700
56
5202
58
2337
56
14309
61
2337
56
14309
61
430
53
14309
61
4482
56
2337
56
4533
56
2337
56
14309
61
4482
56
4517
56
5202
58
5202
58
14309
61
14309
61
14309
61
4482
56
4448
57
14309
61
4448
57
1352
56
14309
61
2337
56
4482
56
4448
57
14309
61
5202
58
14309
61
14309
61
805
56
2221
56
4471
56
528
56
4448
57
14309
61
4482
56
4517
56
14309
61
14309
61
629
56
14309
61
14309
61
4471
56
4482
56
14309
61
1537
56
1095
56
578
55
4517
56
4517
56
4448
57
14309
61
4471
56
4517
56
14309
61
14309
61
427
55
14309
61
14309
61
4533
56
14309
61
2337
56
1163
56
4482
56
4448
57
4471
56
4517
56
14309
61
4482
56
1537
56
4482
56
4533
56
911
56
4533
56
14309
61
14309
61
14309
61
2337
56
4448
57
4533
56
5202
58
4482
56
1537
56
14309
61
4533
56
4471
56
4533
56
4517
56
14309

61
14309
61
14309
61
14309
61
14309
61
14309
61
1240
56
4517
56
4533
56
14309
61
14309
61
14309
61
14309
61
5202
58
14309
61
5202
58
4533
56
4448
57
4533
56
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
1042
56
14309
61
14309
61
14309
61
1048
56
14309
61
14309
61
14309
61
14309
61
731
56
14309
61
4517
56
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
4517
56
14309
61
4471
56
14309
61
14309
61
4448
57
4533
56
14309
61
14309
61
14309
61
14309
61
14309
61
4533
56
4471
56
1157
56
4471
56
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
1095
56
14309
61
14309
61
1048
56
14309
61
14309
61
4517
56
14309
61
505
56
4517
56
4517
56
14309
61
1095
56
4517
56
4471
56
14309
61
2221
56
5202
58
14309
61
4471
56
14309
61
14309
61
1157
56
14309
61
5202
58
463
54
14309
61
2337
56
4517
56
14309
61
14309
61
1163
56
4448
57
14309
61
5202
58
14309
61
14309
61
4517
56
14309
61
14309
61
14309
61
1352
56
14309
61
14309
61
14309
61
5202
58

14309
61
14309
61
5202
58
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
4471
56
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
4533
56
4482
56
629
56
14309
61
14309
61
14309
61
5202
58
4533
56
4448
57
14309
61
1163
56
4471
56
14309
61
4533
56
14309
61
4517
56
4482
56
14309
61
14309
61
4471
56
4482
56
14309
61
14309
61
5202
58
14309
61
1048
56
14309
61
2221
56
5202
58
14309
61
4448
57
4517
56
4471
56
14309
61
14309
61
14309
61
14309
61
14309
61
14309
61
4517
56
14309
61
14309
61
14309
61


KeyboardInterrupt: 

In [50]:
Ln, Lp, G, N = sample(G.copy(), 0.1)

index = len(Ln)
predicted = predict(G, Ln, Lp)

In [7]:
G = nx.Graph(nx.read_pajek("data/circles.net")).to_undirected()
AUC_runs(G, "Facebook", 10)

Gnutella:
  Leiden: 0.9557350107673127
  Preferential attachment: 0.8314065510597303
  Adamic adar index: 0.9929049076277909


In [8]:
G = nx.Graph(nx.read_pajek("data/nec.net")).to_undirected()
AUC_runs(G, "Internet", 10)

Internet:
  Leiden: 0.8994864403459182
  Preferential attachment: 0.8232417228736951
  Adamic adar index: 0.6972222999636171


## 4. Get at least 70% right!

In [20]:
def read_and_split(filepath):
    G = nx.Graph() 

    with open(filepath, 'r') as f:
        f.readline()
        test = []
        for line in f:
            if line.startswith("*"):
                continue

            l = line.split()
            if len(l) == 3:
                if l[1][-3:-1] == '13':
                    G.add_node(int(l[0]))
                    test.append([int(l[0]), l[2]])
                else:
                    G.add_node(int(l[0]), label=l[2])
            else:
                G.add_edge(int(l[0]), int(l[1]))

    return G, test

In [23]:
G, test = read_and_split("data/aps_2008_2013.net")
predicted = node_classification.harmonic_function(G)

count = 0
for (node, label) in test:
    if predicted[node-1] == label:
        count += 1

acc = count / len(test)
acc

MemoryError: Unable to allocate 50.0 GiB for an array with shape (74467, 90162) and data type float64

In [24]:
def build_synhtetic_graph(mu = 0.5):
    G = nx.Graph(name = "girvan_newman")
    n = 72
    cluster_div = 24
    for i in range(n):
        G.add_node(i, cluster = i // cluster_div + 1)
    for i in range(n):
        for j in range(i + 1, n):
            if G.nodes[i]['cluster'] == G.nodes[j]['cluster']:
                if random.random() < 720 * (1 - mu) / (cluster_div-1):
                    G.add_edge(i, j)
            else:
                if random.random() < 720 * mu / (n-cluster_div):
                    G.add_edge(i, j)
    return G

In [27]:
G = build_synhtetic_graph(0)

In [34]:
2*len(list(G.edges()))/72

20.52777777777778

In [38]:
nx.connected_components(G)

<generator object connected_components at 0x0000019926BCD6C8>

In [40]:
NodeClustering([[r] for r in range(10)], graph=G, method_name="rand_graph")